# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

pageurl = 'http://books.toscrape.com/'
page = requests.get(pageurl)
soup = bs(page.content, 'html.parser')

In [2]:
# function that finds our content block which includes both the book info and the Next button
def find_block(soup):
    selection = soup.find('div', class_="alert alert-warning")
    books = selection.nextSibling.nextSibling 
    return books

In [3]:
# function that gets the titles of the books
def retrieve_titles(books):
    titles = books.findAll('h3')
    alltitles = [item.find('a').attrs['title'] for item in titles]
    return alltitles

In [4]:
# function that gets the ratings of the books
def retrieve_ratings(books):
    ratings = []
    regex = re.compile("star-rating (.*)")
    raw_ratings = books.findAll('p', {"class" : regex})
    for rating in raw_ratings:
        ratings.append(rating['class'][1]) 
    conversion = {'One' : 1, 'Two' : 2, 'Three' : 3, 'Four' : 4, 'Five' : 5}
    ratings = [conversion[x] for x in ratings]
    return ratings


In [5]:
# function that gets the prices of the books
def retrieve_prices(books):
    prices = [price.text for price in books.findAll('p', class_="price_color")]
    prices = [float(p[1:]) for p in prices]
    return prices
    

In [6]:
# function that gets the availability of the books
def retrieve_availabilities(books):
    availability = [item.text.strip() for item in books.findAll('p', class_="instock availability")]
    return availability

In [7]:
# script that moves to the next page using the Next button
# There's a conditional because the Next button changes after the first page
def scrape_it(books, pageurl):
    if 'catalogue/' in (books.find('li', class_='next').find('a').get('href')):
        URL = pageurl + (books.find('li', class_='next').find('a').get('href'))
    else:
        URL = pageurl + 'catalogue/' + (books.find('li', class_='next').find('a').get('href'))
    page = requests.get(URL)
    soup = bs(page.content, 'html.parser')
    return soup



In [8]:
books = find_block(soup)
(books.find('li', class_='next').find('a').get('href'))

if 'catalogue/' in (books.find('li', class_='next').find('a').get('href')):
    print('yes')
else:
    print('no')

yes


## Process in Plain English

Have lists made for each data set we are tracking.

Run the entire process on the page, extracting all of the information for each page before moving on.

Add the current information to the existing lists, so that the lists keep getting more info added.

Move on to the next page, until the script can't find a "Next" button to press.
If it DOES find a "Next" button, run the script to open the new url, and restart the scrape for that page.
If there is no next button, break out of the script and print the results.

Next, we will load the results into a Pandas dataframe.

In [9]:
titles = []
ratings = []
prices = []
availabilities = []

In [10]:
titles = []
ratings = []
prices = []
availabilities = []

count = 1

for count in range(1,51):
    books = find_block(soup)
    titles = titles + retrieve_titles(books)
    ratings = ratings + retrieve_ratings(books)
    prices = prices + retrieve_prices(books)
    availabilities = availabilities + retrieve_availabilities(books)
    count = count+1
#    if (books.find('li', class_='next'):
#        soup = scrape_it(books, pageurl)
#        continue
#    else:
#        break


books = find_block(soup)
titles = titles + retrieve_titles(books)
ratings = ratings + retrieve_ratings(books)
prices = prices + retrieve_prices(books)
availabilities = availabilities + retrieve_availabilities(books)
soup = scrape_it(books, pageurl)

In [11]:
df = pd.DataFrame([titles, ratings, prices, availabilities]).transpose()
df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
df.tail()

,Title,Star_Rating,Price_(pounds),Availability
995,Our Band Could Be Your Life: Scenes from the A...,3,57.25,In stock
996,Olio,1,23.88,In stock
997,Mesaerion: The Best Science Fiction Stories 18...,1,37.59,In stock
998,Libertarianism for Beginners,2,51.33,In stock
999,It's Only the Himalayas,2,45.17,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [12]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!